In [1]:
import sys
sys.path.append("../../")
from radarange_orchestrator import llm

In [2]:
from radarange_orchestrator.llm_backend import LLM_Config
from radarange_orchestrator.tools import time_tool

# m = llm(model='qwq-32b@q4_k_m', backend='lmstudio')
m = llm(model='QwQ-32B-Q4_K_M.gguf', backend='llama_cpp', config=LLM_Config(gpus=[1], ctx_size=10000))
chat = m.chat(tools=[time_tool])

llama_context: n_ctx_per_seq (10000) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


In [3]:
message = """What time is it?"""

In [4]:
chat.add_user_message(message)
# print(f"Your message is {m.model.model.model.count_tokens(message)} tokens")
m.act(chat, max_tokens_per_message = 10000, temperature=0.8, on_message=chat.append)
chat.show_final_answer()

history=[UserMessage(role='user', content='What time is it?')] tools=[Tool(name='CurrentTime', description='Gets actual current time in local timezone', func=<function get_time at 0x7a9acf03ad40>)] CODE_EXTS=['.h', '.hpp', '.c', '.cpp', '.cu', '.txt', '.proto', '.py']
tool_calls [{'name': 'CurrentTime', 'args': {'tool_input': ''}, 'id': 'CurrentTime_1', 'type': 'tool_call'}]
history=[UserMessage(role='user', content='What time is it?'), AssistantMessage(role='assistant', content='<think>\nOkay, the user is asking, "What time is it?" I need to figure out which tool to use here. Let me check the tools provided. There\'s a function called CurrentTime that gets the actual current time in the local timezone. The parameters require a tool_input, which is a string. \n\nWait, the function\'s parameters have a tool_input, but the user didn\'t specify a timezone. Maybe the function defaults to the local timezone of the user? The description says it\'s the local timezone, so maybe the tool_input 

TypeError: Object of type Undefined is not JSON serializable

In [5]:
from copy import deepcopy

from llama_cpp import ChatCompletionRequestAssistantMessage
from radarange_orchestrator.types.history import AssistantMessage
from radarange_orchestrator.types.llama_cpp_bindings import to_llama_tool, to_llama_tool_call

def to_llama_assistant_message(message: AssistantMessage) -> ChatCompletionRequestAssistantMessage:
    return ChatCompletionRequestAssistantMessage(
        role='assistant',
        content=message.content,
        tool_calls=[to_llama_tool_call(tc) for tc in message.tool_calls],
        finish_reason=message.finish_reason
    )

nchat = deepcopy(chat)
nchat.history[1] = to_llama_assistant_message(nchat.history[1])

response = m.model.model.llm.create_chat_completion(
    nchat,
    tools=[to_llama_tool(time_tool)],
)
response['choices'][0]['message']['content']

"<think>\nOkay, the user is asking for the current time. I need to use the CurrentTime function provided. The function requires a tool_input parameter, but looking at the example, it seems like it can be an empty string. Let me check the function description again. It says it gets the actual current time in the local timezone. The parameters are a object with tool_input as a string. Since the user didn't specify a particular timezone, maybe the function defaults to the local one. I'll call CurrentTime with an empty string as the argument. That should get the current time. I'll make sure the JSON is correctly formatted to avoid errors.\n</think>\n\nThe current time is <current_time>."

In [6]:
from radarange_orchestrator.chat import Chat

c1 = Chat(message)
r1 = m.model.model.llm.create_chat_completion(
    c1,
    tools=[to_llama_tool(time_tool)],
)
print(r1['choices'][0]['message']['content'])

<think>
Okay, the user is asking "What time is it?" so they want to know the current time. I need to use the CurrentTime function provided. The function requires a tool_input parameter, but the user didn't specify any particular location. Hmm, maybe the tool just needs a placeholder. Let me check the function description again. It says it gets the actual current time in the local timezone. Wait, does the user's local timezone need to be specified here? The parameters for the function have a tool_input of type string. Maybe I should assume the user's timezone, but since I don't have that info, perhaps I can just call the function with an empty string or a default. Alternatively, maybe the function can handle it without additional input. The description mentions "local timezone" so maybe the function is set up to get the local time based on the server's settings. I'll proceed by calling CurrentTime with an empty tool_input or just a placeholder like "user's local timezone". Let me struct

In [7]:
from radarange_orchestrator.utils.extract_tool_calls import extract_tool_calls


tool_calls = extract_tool_calls(r1['choices'][0]['message']['content'])
finish_reason = r1['choices'][0]['finish_reason'] if len(tool_calls) == 0 else 'tool_call'

rp1 = AssistantMessage(
    role='assistant',
    content=r1['choices'][0]['message']['content'],
    tool_calls=tool_calls,
    finish_reason=finish_reason
)
c1.append(rp1)

In [8]:
print(rp1.tool_calls)
t1 = time_tool.run(tool_input=rp1.tool_calls[0]['args'], tool_call_id=rp1.tool_calls[0]['id'])
print(t1)
c1.append(t1)

[{'name': 'CurrentTime', 'args': {'tool_input': "user's local timezone"}, 'id': 'CurrentTime_2', 'type': 'tool_call'}]
content='2025-07-09 01:34:00' name='CurrentTime' tool_call_id='CurrentTime_2'


In [9]:
c_fixed = [
    c1[0],
    to_llama_assistant_message(c1[1]),
    c1[2]
]

In [10]:
r2 = m.model.model.llm.create_chat_completion(
    c_fixed,
    tools=[to_llama_tool(time_tool)],
)
print(r2['choices'][0]['message']['content'])

<think>
Okay, the user is asking for the current time. I need to use the CurrentTime function. The function requires the user's local timezone, but they didn't specify which one. I should call the function with the placeholder "user's local timezone" as instructed. Wait, the example in the tools shows that the tool_input is a string. Maybe I should just pass that exact phrase. Let me check the function parameters again. The tool_input is a string, so I'll use that. I'll make the tool call with that argument. Once I get the response, I can format the answer to show the current time in their timezone.
</think>

The current time in your local timezone is: **<current_time>**. 

Please note that I couldn't directly detect your timezone, so you might need to specify your location for more accurate results.


In [ ]:
from radarange_orchestrator.tools import ToolCall, InvalidToolCall
from radarange_orchestrator.types.history import AssistantMessage
from langchain_core.messages import ToolMessage

tc = ToolCall(
    {'name': 'CurrentTime', 'args': {}, 'id': 'CurrentTime_1', 'type': 'tool_call'}
)

ma = AssistantMessage(
    role='assistant',
    content='kek',
    tool_calls=[tc],
    finish_reason='tool_call',
)

ToolMessage(
    content='Tool call error: Too many arguments to single-input tool CurrentTime.\n                Consider using StructuredTool instead. Args: []\nTraceback: <traceback object at 0x72521022c180>',
    tool_call_id='',
    status='success',
)

ToolMessage(content='Tool call error: Too many arguments to single-input tool CurrentTime.\n                Consider using StructuredTool instead. Args: []\nTraceback: <traceback object at 0x72521022c180>', tool_call_id='')

In [ ]:
from radarange_orchestrator.types.llama_cpp_bindings import to_llama_tool


tools = [to_llama_tool(time_tool)]
response = m.model.model.llm.create_chat_completion(
    message,
    tools=tools,
)

In [ ]:
import json
print(json.dumps(response, indent=2))
print(response['choices'][0]['message']['content'])

{
  "id": "chatcmpl-8a183599-b5cb-4aa6-bd49-ff7dc2455e41",
  "object": "chat.completion",
  "created": 1752009789,
  "model": "/root/radarange-orchestrator/radarange_orchestrator/utils/../../models/QwQ-32B-Q4_K_M.gguf",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "<think>\nOkay, the user asked for the current time and date. Let me check the tools available. There's a function called CurrentTime that does exactly that. It doesn't require any parameters, so I just need to call it. I'll make sure to format the tool call correctly within the XML tags. Alright, I'll return the function name and an empty arguments object since there's nothing needed here.\n</think>\n\n<tool_call>\n{\"name\": \"CurrentTime\", \"arguments\": {}}\n</tool_call>"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 125,
    "completion_tokens": 100,
    "total_tokens": 225
  }
}
<think>
Okay, the u

In [ ]:
prompt = "And the weather is good, isn't it?"
chat.add_user_message(prompt)
m.act(chat, max_tokens_per_message = 10000, temperature=0.8, on_message=chat.append)
chat.show_final_answer()

got tools from call [] and chat tools [Tool(name='CurrentTime', description='Shows current time and date.', func=<function get_time at 0x79eb3e53edd0>, zero_args=True)]
act got response role='assistant' content='<think>\nOkay, the user just asked, "And the weather is good, isn\'t it?" Let me think about how to respond.\n\nFirst, looking at the tools provided earlier, there\'s only the CurrentTime function. The user is now asking about the weather, which isn\'t covered by the available tools. So I can\'t use a function call here. \n\nI need to let the user know that I don\'t have access to weather information. But I should do it politely. Maybe say something like, "I don\'t have access to current weather data..." and offer to help with something else. \n\nWait, the user might be expecting a follow-up after the time question. Since I can\'t answer the weather part, I should acknowledge their statement and redirect. Let me check the response structure again. The user\'s previous question 



I don't have access to current weather data or the ability to check real-time weather conditions. My capabilities are limited to the tools provided, which currently only include showing the current time and date. Let me know if you'd like me to share the current time again or help with something else!